In [1]:
import tensorflow as tf
import numpy as np

C:\Users\Natielle\anaconda3\envs\tensorflow2_py37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Natielle\anaconda3\envs\tensorflow2_py37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Natielle\anaconda3\envs\tensorflow2_py37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Natielle\anaconda3\env


## Aplicando a rede neural para simular o operador AND e vamos aplicar a função de ativação step

1 e 1 = 1

1 e 0 = 0

0 e 1 = 0

0 e 0 = 0


In [39]:
# definindo os dados
X = np.array([[0.0, 0.0],
              [0.0, 1.0],
              [1.0, 0.0],
              [1.0, 1.0]])

Y = np.array([[0.0], [0.0], [0.0], [1.0]])

print("Atributos: \n", X)
print("Labels: \n", Y)

Atributos: 
 [[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
Labels: 
 [[0.]
 [0.]
 [0.]
 [1.]]


In [10]:
# Definindo os pesos das entradas
W = tf.Variable(tf.zeros([2, 1], dtype = tf.float64)) # é uma variável pois ela será alterada em execução
# tf.zeros([2, 1], dtype = tf.float64) -> é uma matriz de 2 linhas e 1 coluna pois é um peso pra cada entrada x1 e x2
print("Tipo da variável pesos: ", type(W))

# Observando os valores da variável peso
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("Matriz dos pesos:\n", sess.run(W))

Tipo da variável pesos:  <class 'tensorflow.python.ops.variables.RefVariable'>
Matriz dos pesos:
 [[0.]
 [0.]]


In [11]:
# Agora vamos definir a camada de saida
camada_saida = tf.matmul(X, W) # estamos multiplicando as entradas pelos pesos

# Observando os valores da multiplicacao
with tf.Session() as sess:
    sess.run(init)
    print("Multiplicação:\n", sess.run(camada_saida))

Multiplicação:
 [[0.]
 [0.]
 [0.]
 [0.]]


Como W é, inicialmente, zero faz sentido o resultado da multiplicação ser zeros.

In [12]:
# implementando a função de ativação step. 
def step(x):
    # tf.math.greater_equal(x, 1) -> retorna true se x > 1 e retorna false se x < 1
    # tf.to_float -> converte o true em 1.0 e converte false em 0.0
    return tf.cast(tf.to_float(tf.math.greater_equal(x, 1)), tf.float64)

In [13]:
# Aplicando a função de ativação step à camada de saída
camada_saida_ativacao = step(camada_saida)

# Observando os valores da aplicacao
with tf.Session() as sess:
    sess.run(init)
    print("Aplicação:\n", sess.run(camada_saida_ativacao))

Aplicação:
 [[0.]
 [0.]
 [0.]
 [0.]]


In [15]:
# Ao aplicarmos a função de ativação na camada de saída, vamos calcular o erro agora
erro = tf.subtract(Y, camada_saida_ativacao) # erro = label_real - label_previsto

# Observando os valores encontrados de erro
with tf.Session() as sess:
    sess.run(init)
    print("Y:\n", Y)
    print("\ncamada_saida_ativacao:\n", sess.run(camada_saida_ativacao))
    print("\nErros:\n", sess.run(erro))

Y:
 [[0.]
 [0.]
 [0.]
 [1.]]

camada_saida_ativacao:
 [[0.]
 [0.]
 [0.]
 [0.]]

Erros:
 [[0.]
 [0.]
 [0.]
 [1.]]


In [16]:
# Agora vamos computar o delta, que vai permitir que encontremos o mínimo global depois de algumas rodadas
# Ou seja, permite que otimizemos o erro de modo que seja o menor possível
delta = tf.matmul(X, erro, transpose_a = True) # transpose_a -> significa que vai fazer a transposta de X 

In [18]:
# Agora vamos definir os pesos da próxima rodada
taxa_aprendizado = 0.1
treinamento = tf.assign(W, tf.add(W, tf.multiply(delta, taxa_aprendizado))) 
# tf.assign -> atualiza os valores dos pesos

# Observando uma simulação da atualização dos pesos
with tf.Session() as sess:
    sess.run(init)
    print("W:\n", sess.run(W))
    print("\nW com novos pesos:\n", sess.run(treinamento))

W:
 [[0.]
 [0.]]

W com novos pesos:
 [[0.1]
 [0.1]]


Agora que já fizemos o processo de treinamento uma vez, vamos repeti-lo N vezes e cada vez que esse processo é executado dizemos que temos N épocas.


In [30]:
# Executando várias épocas

# mas antes vamos ver a diferença na sintaxe
with tf.Session() as sess:
    sess.run(init)
    for i in range(2):
        erro_total, _ = sess.run([erro, treinamento])
        print("\n---------- para i = ", i)
        print("erro_total:")
        print(erro_total)
        print("\n")
        print("treinamento (novos pesos):")
        print(_)
        



---------- para i =  0
erro_total:
[[0.]
 [0.]
 [0.]
 [1.]]


treinamento (novos pesos):
[[0.1]
 [0.1]]

---------- para i =  1
erro_total:
[[0.]
 [0.]
 [0.]
 [1.]]


treinamento (novos pesos):
[[0.2]
 [0.2]]


In [36]:
# agora sim vamos executar várias épocas
with tf.Session() as sess:
    sess.run(init)
    epoca = 0
    for i in range(15):
        epoca += 1
        erro_total, _ = sess.run([erro, treinamento])
        erro_soma = tf.reduce_sum(erro_total) # somando os erros individuais
        # print("Erro total: ", erro_total)
        print('Época:', epoca, ' Erros somados: ', sess.run(erro_soma))
        
        # Vamos executar as épocas até que o erro seja zero
        if erro_soma.eval() == 0.0:
            break
            
    W_final = sess.run(W)

Época: 1  Erros somados:  1.0
Época: 2  Erros somados:  1.0
Época: 3  Erros somados:  1.0
Época: 4  Erros somados:  1.0
Época: 5  Erros somados:  1.0
Época: 6  Erros somados:  0.0


In [35]:
# Podemos ver que os pesos finais serão
print(W_final)

[[0.5]
 [0.5]]


Caso queiramos testar a rede, fazemos isso:

In [37]:
camada_saida_teste = tf.matmul(X, W_final) # Usasmos W_final pois são os pesos que a rede otimizou 
camada_saida_ativacao_teste = step(camada_saida_teste) # aplica a função de ativação 

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(camada_saida_ativacao_teste))

[[0.]
 [0.]
 [0.]
 [1.]]


Podemos ver que a rede otimizou os pesos de modo que não houvesse erros (nesse caso). 
Podemos confirmar isso com o Y, que são os valores esperados.


In [38]:
print(Y)

[[0.]
 [0.]
 [0.]
 [1.]]


## Aplicando a rede neural para simular o operador OR e vamos aplicar a função de ativação step


1 e 1 = 1

1 e 0 = 1

0 e 1 = 1

0 e 0 = 0


In [41]:
# definindo os dados
X = np.array([[0.0, 0.0],
              [0.0, 1.0],
              [1.0, 0.0],
              [1.0, 1.0]])

Y = np.array([[0.0], [1.0], [1.0], [1.0]])

print("Atributos: \n", X)
print("Labels: \n", Y)

Atributos: 
 [[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
Labels: 
 [[0.]
 [1.]
 [1.]
 [1.]]


Agora vamos setar inicialmente e arbritariamente os pesos para fazer os calculos iniciais e depois aplicamos as épocas para encontramos os pesos ideais.

In [45]:
# Definindo os pesos das entradas
W = tf.Variable(tf.zeros([2, 1], dtype = tf.float64))

# Agora vamos definir a camada de saida
camada_saida = tf.matmul(X, W)

# Aplicando a função de ativação step à camada de saída
camada_saida_ativacao = step(camada_saida)

# Vamos calcular o erro agora
erro = tf.subtract(Y, camada_saida_ativacao)

# Encontrando o delta para ajudar na otimização
delta = tf.matmul(X, erro, transpose_a = True)

# Agora vamos definir os pesos da próxima rodada
taxa_aprendizado = 0.1
treinamento = tf.assign(W, tf.add(W, tf.multiply(delta, taxa_aprendizado)))

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    epoca = 0
    for i in range(15):
        epoca += 1
        erro_total, _ = sess.run([erro, treinamento])
        erro_soma = tf.reduce_sum(erro_total) # somando os erros individuais
        print('Época:', epoca, ' Erros somados: ', sess.run(erro_soma))
        
        if erro_soma.eval() == 0.0: # Vamos executar as épocas até que o erro seja zero
            break
            
    W_final = sess.run(W)
    print("\nPesos finais: \n", W_final)

Época: 1  Erros somados:  3.0
Época: 2  Erros somados:  3.0
Época: 3  Erros somados:  3.0
Época: 4  Erros somados:  2.0
Época: 5  Erros somados:  2.0
Época: 6  Erros somados:  2.0
Época: 7  Erros somados:  2.0
Época: 8  Erros somados:  0.0

Pesos finais: 
 [[1.]
 [1.]]


Agora para vermos como a rede se saiu, fazemos:

In [46]:
camada_saida_teste = tf.matmul(X, W_final) # Usasmos W_final pois são os pesos que a rede otimizou 
camada_saida_ativacao_teste = step(camada_saida_teste) # aplica a função de ativação 

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(camada_saida_ativacao_teste))

[[0.]
 [1.]
 [1.]
 [1.]]


Portanto, podemos notar que, de fato, é correspondente ao operador OR! O que era esperado.